# Metadata Analysis

In [2]:
import pandas as pd
import numpy as np
import re
%load_ext autoreload
%autoreload 2
from cord.cord19 import ResearchPapers
from pathlib import Path, PurePath
from IPython.display import display

pd.options.display.max_colwidth=200
pd.options.display.max_rows = 4000

In [3]:
data_path = Path('data') / 'CORD-19-research-challenge'
metadata_path = PurePath(data_path) / 'metadata.csv'

## Load Metadata

In [ ]:
metadata = ResearchPapers.from_data_dir()

Loading metadata from data\CORD-19-research-challenge
Cleaning metadata

Indexing research papers
Creating the BM25 index from the abstracts of the papers
Use index="text" if you want to index the texts of the paper instead


In [19]:
from cord.cord19 import *

metadata = pd.read_csv(metadata_path,
                               dtype={'Microsoft Academic Paper ID': str,
                                      'pubmed_id': str})
metadata = metadata.pipe(start) \
                   .pipe(clean_title) \
                   .pipe(clean_abstract) 
                   .pipe(fix_dates)
metadata.publish_time.str.contains('Spring').any()

False

In [6]:
research_papers = ResearchPapers.from_data_dir()

Loading metadata from data\CORD-19-research-challenge
Cleaning metadata

Indexing research papers
Creating the BM25 index from the abstracts of the papers
Use index="text" if you want to index the texts of the paper instead
Finished Indexing in 34.0 seconds


In [5]:
from cord.core import describe_dataframe

describe_dataframe(metadata)

,non-null,null,unique,duplicate,most common
cord_uid,45774,0,45773,1,c4u0gxp5
sha,31753,14021,31744,9,4644c32551fb23aa873a7738ecc8d777bd49877e
source,45774,0,6,45768,PMC
title,45774,0,43496,2278,
doi,45774,0,42440,3334,
pmcid,26243,19531,26243,0,PMC4577199
pubmed_id,34641,11133,34639,2,20161146
license,45774,0,13,45761,els-covid
abstract,45774,0,44134,1640,
published,45765,9,6341,39424,2020-01-01 00:00:00


In [8]:
research_papers.search('coronavirus', display='table')

,title,abstract,when,authors
0,"Antigenic relationships among proteins of bovine coronavirus, human respiratory coronavirus OC43, and mouse hepatitis coronavirus A59.",Antisera prepared against each of three single and one pair of major structural proteins of the bovine coronavirus (Mebus strain) were used in immunoblotting studies to measure cross-reactivity wi...,36 years ago,"Hogue, B G; King, B; Brian, D A"
1,Chapter 24 Coronaviridae,"Coronavirus and coronavirus-like infections are described in swine, cattle, horses, cats, dogs, rats, birds, bats, rabbits, ferrets, mink, and various wildlife species, although many coronavirus...",8 years ago,
2,Comparative features of a coronavirus isolated from a cheetah with feline infectious peritonitis,A coronavirus which was isolated from a cheetah (Acinonyx jubatus) that succumbed to feline infectious peritonitis was characterized in vitro. The virus was determined to be highly cell-associate...,31 years ago,"Evermann, James F.; Heeney, Jonathan L.; McKeirnan, Alison J.; O'Brien, Stephen J."
3,Discovery of Seven Novel Mammalian and Avian Coronaviruses in the Genus Deltacoronavirus Supports Bat Coronaviruses as the Gene Source of Alphacoronavirus and Betacoronavirus and Avian Coronavirus...,"Recently, we reported the discovery of three novel coronaviruses, bulbul coronavirus HKU11, thrush coronavirus HKU12, and munia coronavirus HKU13, which were identified as representatives of a nov...",8 years ago,"Woo, Patrick C. Y.; Lau, Susanna K. P.; Lam, Carol S. F.; Lau, Candy C. Y.; Tsang, Alan K. L.; Lau, John H. N.; Bai, Ru; Teng, Jade L. L.; Tsang, Chris C. C.; Wang, Ming; Zheng, Bo-Jian; Chan, Kwo..."
4,Molecular characterization of a novel coronavirus associated with epizootic catarrhal enteritis (ECE) in ferrets,"A novel coronavirus, designated as ferret enteric coronavirus (FECV), was identified in feces of domestic ferrets clinically diagnosed with epizootic catarrhal enteritis (ECE). Initially, partial...",14 years ago,"Wise, Annabel G.; Kiupel, Matti; Maes, Roger K."
5,A novel human coronavirus: Middle East respiratory syndrome human coronavirus,"In 2012, a novel coronavirus, initially named as human coronavirus EMC (HCoV-EMC) but recently renamed as Middle East respiratory syndrome human coronavirus (MERS-CoV), was identified in patients ...",7 years ago,"Geng, HeYuan; Tan, WenJie"
6,Coronavirus Infection in Cats,Cats are susceptible to natural infection with several strains of feline coronavirus that result in either effusive and noneffusive feline infectious peritonitis or enteritis. Excretion of coronav...,27 years ago,"Hoskins, Johnny D."
7,Coronavirus Genomics and Bioinformatics Analysis,The drastic increase in the number of coronaviruses discovered and coronavirus genomes being sequenced have given us an unprecedented opportunity to perform genomics and bioinformatics analysis on...,10 years ago,"Woo, Patrick C. Y.; Huang, Yi; Lau, Susanna K. P.; Yuen, Kwok-Yung"
8,Coronavirus infection of spotted hyenas in the Serengeti ecosystem,"Sera from 38 free-ranging spotted hyenas (Crocuta crocuta) in the Serengeti ecosystem, Tanzania, were screened for exposure to coronavirus of antigenic group 1. An immunofluorescence assay indica...",16 years ago,"East, Marion L; Moestl, Karin; Benetka, Viviane; Pitra, Christian; Höner, Oliver P; Wachter, Bettina; Hofer, Heribert"
9,Physiological and molecular triggers for SARS-CoV membrane fusion and entry into host cells,"During viral entry, enveloped viruses require the fusion of their lipid envelope with host cell membranes. For coronaviruses, this critical step is governed by the virally-encoded spike (S) prote...",2 years ago,"Millet, Jean Kaoru; Whittaker, Gary R."


## Metadata Utility Functions

In [304]:
def describe_column(series):
    col_counts = series.describe().loc[['count', 'unique','top']]
    col_counts.loc['null'] = series.isnull().sum()
    col_counts['duplicate'] = series.dropna().duplicated().sum()
    df = col_counts.to_frame().T
    df = df[['count', 'null', 'unique', 'duplicate','top']] \
                .rename(columns={'count' : 'non-null', 'top':'most common'}).T
    return df

def describe_columns(df, columns=None):
    columns = columns or df.columns
    column_descs = [describe_column(df[col]).T for col in columns]
    return pd.concat(column_descs)

def show_common(data, column, head=20):
    common_column = data[column].value_counts().to_frame()
    common_column = common_column[common_column[column] > 1]
    return common_column.head(head)

## Describe Metadata

In [266]:
describe_columns(metadata)

,non-null,null,unique,duplicate,most common
sha,28462,15758,28450,12,72a5640aa0c307fbe171ca7ad55d3fda48b53988
source_x,44220,0,7,44213,PMC
title,43996,224,42230,1766,Index
doi,40750,3470,40750,0,10.1016/j.jinf.2020.01.014
pmcid,23319,20901,23319,0,PMC5841229
pubmed_id,22943,21277,22943,0,27185404
license,44220,0,13,44207,els-covid
abstract,35806,8414,35099,707,Unknown
publish_time,34197,10023,7045,27152,2020
authors,41074,3146,39285,1789,"Parry, Jane"


## Common Titles

In [267]:
show_common(metadata, 'title')

,title
Index,348
Subject Index,83
Subject index,76
Author index,69
Contents,68
Articles of Significant Interest Selected from This Issue by the Editors,67
Information for Authors,66
Graphical contents list,36
Table of Contents,29
Infectious disease surveillance update,24


### Common Abstracts

In [268]:
show_common(metadata, 'abstract')

,abstract
Unknown,654
Journal Watch presents a brief description of articles recently published in other journals and thought to be of relevance or interest to the AIC readership. Readers are encouraged to refer to the full article for complete information.,3
"Abstract During the millions of years they have coexisted with their hosts, viruses have learned how to manipulate host immune control mechanisms. Viral gene functions provide an overview of many relevant principles in cell biology and immunology. Our knowledge of viral gene functions must be integrated into virus–host interaction networks to understand viral pathogenesis, and could lead to new anti-viral strategies and the ability to exploit viral functions as tools in medicine.",3
"Resumen La faringoamigdalitis aguda (FAA) en el adulto es una de las enfermedades infecciosas más comunes en la consulta del médico de familia. La etiología más frecuente es viral. Dentro de la etiología bacteriana, el principal agente responsable es Streptococcus pyogenes o estreptococo β-hemolítico del grupo A (EBHGA), causante del 5-30% de los casos. En el manejo diagnóstico, las escalas de valoración clínica para predecir la posible etiología bacteriana, son una buena ayuda para seleccionar a qué pacientes se deben practicar las técnicas de detección rápida de antígeno estreptocócico. Es conocido que, en general, sin estas técnicas se tiende al sobrediagnóstico de FAA estreptocócica, con la consiguiente prescripción innecesaria de antibióticos, muchas veces de amplio espectro. Así, con el manejo de las escalas y la técnica de diagnóstico rápido, elaboramos los algoritmos de manejo de la FAA. Los objetivos del tratamiento son acelerar la resolución de los síntomas, reducir el tiempo de contagio y prevenir las complicaciones supurativas locales y no supurativas. Los antibióticos de elección para el tratamiento de la FAA estreptocócica son penicilina y amoxicilina. La asociación de amoxicilina y clavulánico no está indicada en el tratamiento inicial en la infección aguda. Los macrólidos tampoco son un tratamiento de primera elección; su uso debe reservarse para pacientes con alergia a la penicilina. Es importante en nuestro país adecuar tanto el diagnóstico de la FAA bacteriana y la prescripción de antibióticos a la evidencia científica disponible. La implantación de protocolos de actuación en las farmacias comunitarias puede ser de utilidad para identificar y cribar los casos que no requieran tratamiento antibiótico. Abstract Acute pharyngitis in adults is one of the most common infectious diseases seen in general practitioners’ consultations. Viral aetiology is the most common. Among bacterial causes, the main agent is Streptococcus pyogenes or group A β-haemolytic streptococcus (GABHS), which causes 5%-30% of the episodes. In the diagnostic process, clinical assessment scales can help clinicians to better predict suspected bacterial aetiology by selecting patients who should undergo a rapid antigen detection test. If these techniques are not performed, an overdiagnosis of streptococcal pharyngitis often occurs, resulting in unnecessary prescriptions of antibiotics, most of which are broad spectrum. Consequently, management algorithms that include the use of predictive clinical rules and rapid tests have been set up. The aim of the treatment is speeding up symptom resolution, reducing the contagious time span and preventing local suppurative and non-suppurative complications. Penicillin and amoxicillin are the antibiotics of choice for the treatment of pharyngitis. The association of amoxicillin and clavulanate is not indicated as the initial treatment of acute infection. Neither are macrolides indicated as first-line therapy; they should be reserved for patients allergic to penicillin. The appropriate diagnosis of bacterial pharyngitis and proper use of antibiotics based on the scientific evidence available are crucial. Using management algorithms can be helpful in identifying and screen

In [269]:
abstract_title = ['abstract', 'title']

## Conditions

In [270]:
# The title is null or short
title_null = metadata.title.isnull()
title_short = metadata.title.fillna('').apply(len) < 30

# Some titles are is short and unrelated to viruses
# This regex keeps some short titles if they seem relevant
_relevant_re_ = '.*vir.*|.*sars.*|.*mers.*|.*corona.*|.*ncov.*|.*immun.*|.*nosocomial.*'
_relevant_re_ = _relevant_re_ +  '.*epidem.*|.*emerg.*|.*vacc.*|.*cytokine.*'
title_relevant = metadata.title.fillna('').str.match(_relevant_re_, case=False)
title_junk = title_short & ~title_relevant

# The abstract is null, duplicate, short or missing
abstract_null = metadata.abstract.isnull()
abstract_duplicate = metadata.abstract.duplicated()
abstract_unknown = metadata.abstract == 'Unknown'
abstract_short = metadata.abstract.fillna('').apply(len) < 20
abstract_missing = abstract_null | abstract_unknown
sha_null = metadata.sha.isnull()
doi_null = metadata.doi.isnull()

## Text Processing 

In [305]:
_abstract_terms_ = '(Publisher|Abstract|Summary|BACKGROUND|INTRODUCTION)'
def remove_common_terms(abstract):
    return re.sub(_abstract_terms_, '', abstract)

def start(data):
    return data.copy()

def clean_title(data):
    # Set junk titles to NAN
    data.loc[title_junk, 'title'] = ''
    return data

def clean_abstract(data):
    # Set unknowns to NAN
    data.loc[abstract_unknown, 'abstract'] = np.nan
    
    # Fill missing abstract with the title
    data.abstract = data.abstract.fillna(data.title)
    
    # Remove common terms like publisher
    data.abstract = data.abstract.fillna('').apply(remove_common_terms)
   
    return data
   
clean_metadata = metadata.pipe(start) \
                         .pipe(clean_title) \
                         .pipe(clean_abstract)

### Common Titles in Clean Metadata

In [306]:
show_common(clean_metadata, 'title')

,title
,3022
Articles of Significant Interest Selected from This Issue by the Editors,67
Infectious disease surveillance update,24
Opportunities from the Center for Perioperative Education,15
PNAS Plus Significance Statements,12
Current Awareness on Comparative and Functional Genomics,10
New nucleotide sequence data on the EMBL File Server.,8
Bibliography of the current world literature,7
Contents of other veterinary journals from Elsevier,6
Viral gastroenteritis,6


### Common Abstracts in Clean Metadata

In [307]:
show_common(clean_metadata, 'abstract')

,abstract
,2531
Articles of Significant Interest Selected from This Issue by the Editors,67
Infectious disease surveillance update,24
Opportunities from the Center for Perioperative Education,15
PNAS Plus Significance Statements,12
Current Awareness on Comparative and Functional Genomics,10
New nucleotide sequence data on the EMBL File Server.,8
Bibliography of the current world literature,7
Contents of other veterinary journals from Elsevier,6
Immunity to infection,5
